In [ ]:
import cv2
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm.notebook import tqdm
import lensfunpy

In [ ]:
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_FOLDER = os.path.join(ROOT_DIR, "data")

In [ ]:
images = glob.glob(f"{os.path.join(DATA_FOLDER, 'camera_calibration')}/*.JPG")

In [ ]:
img_shape = cv2.imread(images[0]).shape[0:2]

In [ ]:
CHECKERBOARD = (7, 9)

In [ ]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

three_d_points = []
two_d_points = []

objectp3d = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objectp3d[0, :, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
img_shape = cv2.imread(images[0]).shape[0:2]
print(img_shape)

for filename in tqdm(images):
    image = cv2.imread(filename)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    # If desired number of corners are
    # found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray_image, CHECKERBOARD, 
                                             cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    # If desired number of corners can be detected then,
    # refine the pixel coordinates and display
    # them on the images of checker board
    if ret == True:
        three_d_points.append(objectp3d)
        # Refining pixel coordinates
        # for given 2d points.
        corners2 = cv2.cornerSubPix(gray_image, corners, (11, 11), (-1, -1), criteria)
        two_d_points.append(corners2)
        # Draw and display the corners
        #image = cv2.drawChessboardCorners(image, CHECKERBOARD, corners2, ret)
        #plt.imshow(undistorted_image)
        #plt.show()
    else:
        print(f"Pattern was not found on image {filename}")
    
    

In [ ]:
# Perform camera calibration by
# passing the value of above found out 3D points (threedpoints)
# and its corresponding pixel coordinates of the
# detected corners (twodpoints)
ret, intrinsic_matrix, distortion, r_vecs, t_vecs = cv2.calibrateCamera(
    three_d_points, two_d_points, img_shape[::-1], None, None
)
# Displaying required output
print("Camera matrix: ")
print(intrinsic_matrix)
print("Distortion coefficient: ")
print(distortion)

In [ ]:
projection_error = 0
for i in range(len(three_d_points)):
    img_points2, _ = cv2.projectPoints(three_d_points[i], r_vecs[i], t_vecs[i], intrinsic_matrix, distortion)
    error = cv2.norm(two_d_points[i], img_points2, cv2.NORM_L2) / len(img_points2)
    projection_error += error

print(f"Total projection error: {projection_error}")

In [ ]:
camera_matrix_path = os.path.join(DATA_FOLDER, "camera_matrix.npy")
camera_matrix_txt_path = os.path.join(DATA_FOLDER, "camera_matrix.txt")
camera_distortion_path = os.path.join(DATA_FOLDER, "camera_distortion.npy")
camera_distortion_txt_path = os.path.join(DATA_FOLDER, "camera_distortion.txt")

In [ ]:
#np.save(camera_matrix_path, intrinsic_matrix)
#np.savetxt(camera_matrix_txt_path, intrinsic_matrix)
#np.save(camera_distortion_path, distortion)
#np.savetxt(camera_distortion_txt_path, distortion)

In [ ]:
intrinsic_matrix = np.load(camera_matrix_path)
distortion = np.load(camera_distortion_path)

In [ ]:
h, w = 3000, 4000
undistorted_matrix, roi = cv2.getOptimalNewCameraMatrix(intrinsic_matrix, distortion, (w,h), 1, (w,h))

In [ ]:
undistorted_camera_matrix_path = os.path.join(DATA_FOLDER, "undistorted_camera_matrix_2K.npy")

In [ ]:
np.save(undistorted_camera_matrix_path, undistorted_matrix)

In [ ]:
image = cv2.imread(images[10])
undistorted_image = cv2.undistort(image, intrinsic_matrix, distortion, None, undistorted_matrix)

#x, y, w, h = roi
#undistorted_image = undistorted_image[y:y+h, x:x+w]
plt.figure(figsize=(30,50))
plt.subplot(1,2,1)
plt.imshow(image)
plt.subplot(1,2,2)
plt.imshow(undistorted_image)
plt.show()

In [ ]:
h, w = 1080, 1920
undistorted_matrix, roi = cv2.getOptimalNewCameraMatrix(intrinsic_matrix, distortion, (w,h), 1, (w,h))

keparoi_image_path = os.path.join(DATA_FOLDER, "keparoi_left_frame.jpg")
image = cv2.imread(keparoi_image_path)
undistorted_image = cv2.undistort(image, intrinsic_matrix, distortion, None, undistorted_matrix)

x, y, w, h = roi
undistorted_image = undistorted_image[y:y+h, x:x+w]
plt.figure(figsize=(30,50))
plt.subplot(1,2,1)
plt.imshow(image)
plt.subplot(1,2,2)
plt.imshow(undistorted_image)
plt.show()

In [ ]:
cam_maker = 'GoPro'
cam_model = 'HERO5 Black'
lens_maker = 'GoPro'
lens_model = 'fixed lens'

db = lensfunpy.Database()
db
cam = db.find_cameras(cam_maker, cam_model)[0]
print(cam)
lens = db.find_lenses(cam, lens_maker, lens_model)[0]
print(lens)

In [ ]:
focal_length = 15.0
aperture = 1.4
distance = 1

mod = lensfunpy.Modifier(lens, cam.crop_factor, w, h)
mod.initialize(focal_length, aperture, distance)

undist_coords = mod.apply_geometry_distortion()
im_undistorted = cv2.remap(image, undist_coords, None, cv2.INTER_LANCZOS4)
plt.imshow(im_undistorted)
plt.show()